In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import accuracy_score

In [ ]:
def load_data(dataset_path):

  data = []
  test_data = []
  test_labels = []
  labels =[]

  with open(dataset_path, 'r') as file:
      for line_no, line in enumerate(file.readlines()):
          if 0 < line_no <= 35887:
            curr_class, set_type, line = line.split(',')
            image_data = np.asarray([int(x) for x in line.split()]).reshape(48, 48)#Creating a list out of the string then converting it into a 2-Dimensional numpy array.
            image_data =image_data.astype(np.uint8)/255.0

            if (set_type.strip() == 'PrivateTest'):

              test_data.append(image_data)
              test_labels.append(curr_class)
            else:
              data.append(image_data)
              labels.append(curr_class)

      test_data = np.expand_dims(test_data, -1)
      test_labels = to_categorical(test_labels, num_classes = 7)
      data = np.expand_dims(data, -1)
      labels = to_categorical(labels, num_classes = 7)

      return np.array(data), np.array(labels), np.array(test_data), np.array(test_labels)

In [ ]:
dataset_path = "icml_face_data.csv"
train_data, train_labels, test_data, test_labels = load_data(dataset_path)

print("Number of images in Training set:", len(train_data))
print("Number of images in Test set:", len(test_data))

Number of images in Training set: 32298
Number of images in Test set: 3589


In [ ]:
cnn = Sequential()

cnn.add(Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1), kernel_regularizer=l2(0.01)))
cnn.add(Conv2D(64, (3, 3), padding='same',activation='relu'))
cnn.add(BatchNormalization())
cnn.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
cnn.add(Dropout(0.5))

cnn.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
cnn.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
cnn.add(BatchNormalization())
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.5))

cnn.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
cnn.add(BatchNormalization())
cnn.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
cnn.add(BatchNormalization())
cnn.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
cnn.add(BatchNormalization())
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.5))

cnn.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
cnn.add(BatchNormalization())
cnn.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
cnn.add(BatchNormalization())
cnn.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
cnn.add(BatchNormalization())
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.5))

cnn.add(Flatten())
cnn.add(Dense(1024, activation='relu'))
cnn.add(Dropout(0.2))
cnn.add(Dense(1024, activation='relu'))
cnn.add(Dropout(0.2))
cnn.add(Dense(7, activation='softmax'))
adam = optimizers.Adam(lr = 0.001)
cnn.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])


lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, patience=6, mode='auto')
checkpointer = ModelCheckpoint('/content/drive/My Drive/Colab Notebooks/Facial Emotion Recognition/Model/weights_facial.hd5', monitor='val_loss', verbose=1, save_best_only=True)

#training
cnn.fit(
    train_data,
    train_labels,
    epochs = 50,
    batch_size = 32,
    validation_split = 0.2,
    shuffle = True,
    callbacks=[lr_reducer, checkpointer, early_stopper]
)

Epoch 1/50
808/808 [==============================] - 36s 34ms/step - loss: 2.2281 - accuracy: 0.2187 - val_loss: 1.7368 - val_accuracy: 0.3028

Epoch 00001: val_loss improved from inf to 1.73679, saving model to /content/drive/My Drive/Colab Notebooks/Facial Emotion Recognition/Model/weights_facial.hd5
INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/Facial Emotion Recognition/Model/weights_facial.hd5/assets
Epoch 2/50
808/808 [==============================] - 27s 34ms/step - loss: 1.7034 - accuracy: 0.3281 - val_loss: 1.5781 - val_accuracy: 0.3704

Epoch 00002: val_loss improved from 1.73679 to 1.57810, saving model to /content/drive/My Drive/Colab Notebooks/Facial Emotion Recognition/Model/weights_facial.hd5
INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/Facial Emotion Recognition/Model/weights_facial.hd5/assets
Epoch 3/50
808/808 [==============================] - 28s 34ms/step - loss: 1.5534 - accuracy: 0.3943 - val_loss: 1.56

In [ ]:
predicted_test_labels = np.argmax(cnn.predict(test_data), axis=1)
test_labels = np.argmax(test_labels, axis=1)
print ("Accuracy score = ", accuracy_score(test_labels, predicted_test_labels))

Accuracy score =  0.6430760657564781


In [ ]:
from keras.models import model_from_json
model_json = cnn.to_json()
with open("/content/drive/My Drive/Colab Notebooks/Facial Emotion Recognition/model_2.json", "w") as json_file:
    json_file.write(model_json)
cnn.save_weights("/content/drive/My Drive/Colab Notebooks/Facial Emotion Recognition/model_2.h5")
print("Saved model to disk")

Saved model to disk
